In [20]:
# !wget https://github.com/SVizor42/ML_Zoomcamp/releases/download/bee-wasp-data/data.zip
# !unzip data.zip

In [21]:
from tensorflow import keras
from keras import models, layers, optimizers
from keras.preprocessing.image import ImageDataGenerator

import numpy as np
import matplotlib.pyplot as plt

## Data Preparation

In [22]:
train_folder = "data/train"
test_folder = "data/test"

## Create CNN model

In [23]:
model = models.Sequential()

In [24]:
"""
Next, create a convolutional layer (Conv2D):
Use 32 filters
Kernel size should be (3, 3) (that's the size of the filter)
Use 'relu' as activation
"""
conv_2d_layer = layers.Conv2D(32, (3,3), activation="relu", input_shape=(150, 150, 3))
model.add(conv_2d_layer)

In [25]:
"""
Reduce the size of the feature map with max pooling (MaxPooling2D)
Set the pooling size to (2, 2)
"""
max_pooling_2d_layer = layers.MaxPooling2D((2,2))
model.add(max_pooling_2d_layer)

In [26]:
"""Turn the multi-dimensional result into vectors using a Flatten layer"""
model.add(layers.Flatten())

"""Next, add a Dense layer with 64 neurons and 'relu' activation"""
model.add(layers.Dense(64, activation='relu'))

"""
Finally, create the Dense layer with 1 neuron - this will be the output
The output layer should have an activation - use the appropriate activation 
for the binary classification case.
"""
# Appropriate activation for binary classification is "sigmoid"
model.add(layers.Dense(1, activation='sigmoid'))

In [27]:
"""
As optimizer use SGD with the following parameters:
SGD(lr=0.002, momentum=0.8)
"""
sgd_optimizer = optimizers.SGD(learning_rate=0.002, momentum=0.8)
loss_for_binary_class = "binary_crossentropy"

model.compile(
    loss=loss_for_binary_class,
    optimizer=sgd_optimizer,
    metrics=['acc']
)

## Question 1

Since we have a binary classification problem, what is the best loss function for us?

### Answer to Q1:
- binary crossentropy

In [28]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 74, 74, 32)        0         
 g2D)                                                            
                                                                 
 flatten_3 (Flatten)         (None, 175232)            0         
                                                                 
 dense_6 (Dense)             (None, 64)                11214912  
                                                                 
 dense_7 (Dense)             (None, 1)                 65        
                                                                 
Total params: 11215873 (42.79 MB)
Trainable params: 11215873 (42.79 MB)
Non-trainable params: 0 (0.00 Byte)
____________

## Question 2

What's the number of parameters in the convolutional layer of our model? You can use the summary method for that.

### Answer to Q2:
- 11214912

In [32]:
"""For the next two questions, use the following data generator for both train and test sets:
ImageDataGenerator(rescale=1./255)
We don't need to do any additional pre-processing for the images.
"""
train_gen = ImageDataGenerator(rescale=1./255)
test_gen = ImageDataGenerator(rescale=1./255)

"""
When reading the data from train/test directories, check the class_mode parameter.
Which value should it be for a binary classification problem? (binary)
Use batch_size=20
Use shuffle=True for both training and test sets.
"""
common_params = {
    "target_size": (150, 150),
    "batch_size": 20,
    "shuffle": True,
    "class_mode": "binary",
}
train_ds = train_gen.flow_from_directory(
    train_folder,
    **common_params,
)

test_ds = test_gen.flow_from_directory(
    test_folder,
    **common_params,
)

Found 3677 images belonging to 2 classes.
Found 918 images belonging to 2 classes.


In [34]:
"""
For training use .fit() with the following params:

model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator
)
"""
history = model.fit(
    train_ds,
    epochs=10,
    validation_data=test_ds,
)

Epoch 1/10
184/184 [==============================] - 34s 184ms/step - loss: 0.3025 - acc: 0.8888 - val_loss: 0.5115 - val_acc: 0.7778
Epoch 2/10
184/184 [==============================] - 34s 185ms/step - loss: 0.2595 - acc: 0.9094 - val_loss: 0.5380 - val_acc: 0.7549
Epoch 3/10
184/184 [==============================] - 34s 184ms/step - loss: 0.2246 - acc: 0.9230 - val_loss: 0.6100 - val_acc: 0.7353
Epoch 4/10
184/184 [==============================] - 34s 185ms/step - loss: 0.1871 - acc: 0.9451 - val_loss: 0.6778 - val_acc: 0.7135
Epoch 5/10
184/184 [==============================] - 35s 189ms/step - loss: 0.1544 - acc: 0.9578 - val_loss: 0.6124 - val_acc: 0.7266
Epoch 6/10
184/184 [==============================] - 35s 191ms/step - loss: 0.1365 - acc: 0.9657 - val_loss: 0.7452 - val_acc: 0.7298
Epoch 7/10
184/184 [==============================] - 31s 166ms/step - loss: 0.1109 - acc: 0.9758 - val_loss: 0.6300 - val_acc: 0.7702
Epoch 8/10
184/184 [==============================] - 3

In [47]:

training_acc = history.history["acc"]
training_loss = history.history["loss"]
print(np.median(training_acc))
print(np.std(training_loss))


0.9617894887924194
0.07820874997477685


## Question 3

What is the median of training accuracy for all the epochs for this model?

* 0.20
* 0.40
* 0.60
* 0.80

### Answer to Q3:
- 0.80  (closest one to the accuracy I got - 0.96)

## Question 4

What is the standard deviation of training loss for all the epochs for this model?

* 0.031
* 0.061
* 0.091
* 0.131

### Answer to Q4:
- 0.091  (closest one to the loss I got - 0.078)

## Data Augmentation

In [50]:
"""
For the next two questions, we'll generate more data using data augmentations. 

Add the following augmentations to your training data generator:

* `rotation_range=50,`
* `width_shift_range=0.1,`
* `height_shift_range=0.1,`
* `zoom_range=0.1,`
* `horizontal_flip=True,`
* `fill_mode='nearest'`
"""
train_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=50,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest',
)
test_gen = ImageDataGenerator(rescale=1./255)


train_ds = train_gen.flow_from_directory(
    train_folder,
    **common_params,
)
test_ds = test_gen.flow_from_directory(
    test_folder,
    **common_params,
)


Found 3677 images belonging to 2 classes.
Found 918 images belonging to 2 classes.


In [51]:
"""
Let's train our model for 10 more epochs using the same code as previously.
> **Note:** make sure you don't re-create the model - we want to continue training the model
we already started training.
"""
history = model.fit(
    train_ds,
    epochs=10,
    validation_data=test_ds,
)

Epoch 1/10
184/184 [==============================] - 29s 155ms/step - loss: 0.5319 - acc: 0.7590 - val_loss: 0.4967 - val_acc: 0.7821
Epoch 2/10
184/184 [==============================] - 30s 163ms/step - loss: 0.4876 - acc: 0.7707 - val_loss: 0.5228 - val_acc: 0.7658
Epoch 3/10
184/184 [==============================] - 30s 165ms/step - loss: 0.4897 - acc: 0.7748 - val_loss: 0.4690 - val_acc: 0.7963
Epoch 4/10
184/184 [==============================] - 31s 169ms/step - loss: 0.4732 - acc: 0.7928 - val_loss: 0.4684 - val_acc: 0.7941
Epoch 5/10
184/184 [==============================] - 29s 155ms/step - loss: 0.4704 - acc: 0.7868 - val_loss: 0.5123 - val_acc: 0.7658
Epoch 6/10
184/184 [==============================] - 28s 152ms/step - loss: 0.4508 - acc: 0.8009 - val_loss: 0.5259 - val_acc: 0.7571
Epoch 7/10
184/184 [==============================] - 28s 153ms/step - loss: 0.4573 - acc: 0.8042 - val_loss: 0.5078 - val_acc: 0.7691
Epoch 8/10
184/184 [==============================] - 2

In [58]:
validating_acc = history.history["val_acc"]
validating_loss = history.history["val_loss"]
last_5_epoch_val_acc = validating_acc[5:]

print(np.mean(validating_loss))
print(np.average(last_5_epoch_val_acc))

0.48288453817367555
0.7880174279212951


## Question 5

What is the mean of test loss for all the epochs for the model trained with augmentations?

* 0.18
* 0.48
* 0.78
* 0.108

### Answer to Q5:
- 0.48

## Question 6

What's the average of test accuracy for the last 5 epochs (from 6 to 10)
for the model trained with augmentations?
* 0.38
* 0.58
* 0.78
* 0.98

### Answer to Q6:
- 0.78